In [ ]:
import os
import cv2
import tkinter as tk
from tkinter import simpledialog
import mediapipe as mp

def capture_and_process(num_images):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
    mp_drawing = mp.solutions.drawing_utils
    
    
    folder_name = "CollectedData"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    
    cap = cv2.VideoCapture(0)
    print("Press 's' to save the cropped hand image with label, or 'q' to quit.")
    
    image_count = 0
    
    while image_count < num_images:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
    
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        
       
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        
        cv2.imshow("Capture Image with Landmarks", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('s') and image_count < num_images:  
          
            root = tk.Tk()
            root.withdraw()
            label = simpledialog.askstring("Input", "Enter the label for this image:")
            root.destroy()
            
            if label and results.multi_hand_landmarks:
                
                label_dir = os.path.join(folder_name, label)
                
                if not os.path.exists(label_dir):
                    os.makedirs(label_dir)
                
                for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                    
                    img_h, img_w, _ = frame.shape
                    x_min, y_min = img_w, img_h
                    x_max, y_max = 0, 0
                    
                    for landmark in hand_landmarks.landmark:
                        x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                        x_min, y_min = min(x, x_min), min(y, y_min)
                        x_max, y_max = max(x, x_max), max(y, y_max)
                    
                    
                    padding = 20
                    x_min = max(0, x_min - padding)
                    y_min = max(0, y_min - padding)
                    x_max = min(img_w, x_max + padding)
                    y_max = min(img_h, y_max + padding)
                    
                    
                    cropped_hand = frame[y_min:y_max, x_min:x_max]
                    
                    
                    file_count = len(os.listdir(label_dir)) + 1 
                    file_name = os.path.join(label_dir, f"{file_count}.jpg")
                    cv2.imwrite(file_name, cropped_hand)
                    print(f"Cropped hand image saved as {file_name}")
                    image_count += 1
                    
                    
                    if image_count >= num_images:
                        break
            else:
                print("No hand detected or label not provided.")
        
        elif key == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()


capture_and_process(num_images=100) 


Press 's' to save the cropped hand image with label, or 'q' to quit.
Cropped hand image saved as CollectedData\hello\2.jpg
Cropped hand image saved as CollectedData\hello\3.jpg


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def load_data(folder_name):
    images = []
    labels = []
    
    for label in os.listdir(folder_name):
        label_dir = os.path.join(folder_name, label)
        for img_file in os.listdir(label_dir):
            img_path = os.path.join(label_dir, img_file)
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image at {img_path} is not valid and will be skipped.")
                continue
            
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
            img = cv2.resize(img, (128, 128)) 
            images.append(img)
            labels.append(label)
    
    images = np.array(images) / 255.0 
    labels = np.array(labels)

   
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    np.save('path_to_your_label_encoder_classes.npy', label_encoder.classes_)
    
    
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    
    return x_train, x_test, y_train, y_test


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def build_model():
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(len(np.unique(y_train)), activation='softmax')  # Number of output classes
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


folder_name = "CollectedData"
x_train, x_test, y_train, y_test = load_data(folder_name)


model = build_model()


history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=32)


Epoch 1/20


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.3333 - loss: 0.7780 - val_accuracy: 1.0000 - val_loss: 8.6425e-04
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.6667 - loss: 2.3039 - val_accuracy: 1.0000 - val_loss: 0.0388
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.6667 - loss: 0.9798 - val_accuracy: 0.0000e+00 - val_loss: 0.7297
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.3333 - loss: 0.6845 - val_accuracy: 1.0000 - val_loss: 0.6495
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - accuracy: 0.6667 - loss: 0.6636 - val_accuracy: 1.0000 - val_loss: 0.4961
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.6667 - loss: 0.6043 - val_accuracy: 1.0000 - val_loss: 0.3682
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.6667 - loss: 0.5589 - val_accuracy: 1.0000 - val_loss: 0.2796
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - accuracy: 0.6667 - loss: 0.5235 - val_accuracy: 1.0000 - val_loss: 0.2418
Epo

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder


label_encoder = LabelEncoder()
label_encoder.classes_ = np.load('path_to_your_label_encoder_classes.npy')  


mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

def predict_gesture():
   
    cap = cv2.VideoCapture(0)
    print("Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture image")
            break
        
       
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                
                img_h, img_w, _ = frame.shape
                x_min, y_min = img_w, img_h
                x_max, y_max = 0, 0
                for landmark in hand_landmarks.landmark:
                    x, y = int(landmark.x * img_w), int(landmark.y * img_h)
                    x_min, y_min = min(x, x_min), min(y, y_min)
                    x_max, y_max = max(x, x_max), max(y, y_max)

                padding = 20
                x_min = max(0, x_min - padding)
                y_min = max(0, y_min - padding)
                x_max = min(img_w, x_max + padding)
                y_max = min(img_h, y_max + padding)
                
                cropped_hand = frame[y_min:y_max, x_min:x_max]
                cropped_hand = cv2.cvtColor(cropped_hand, cv2.COLOR_BGR2RGB) 
                cropped_hand = cv2.resize(cropped_hand, (128, 128))  
                cropped_hand = np.expand_dims(cropped_hand, axis=0) / 255.0  
                
             
                predictions = model.predict(cropped_hand)
                gesture_label_encoded = np.argmax(predictions, axis=1)[0]
                gesture_label = label_encoder.inverse_transform([gesture_label_encoded])[0]
                
                
                cv2.putText(frame, f'Gesture: {gesture_label}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

     
        cv2.imshow("Real-time Hand Gesture Prediction", frame)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Quit
            break

    cap.release()
    cv2.destroyAllWindows()

predict_gesture()


Press 'q' to quit.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


...

c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.G

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.G

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


c:\Users\KIIT\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
